# Multi Layer Percepron
with R kernel

In [14]:
install.packages("tensorflow")


Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/kj/dkjqkk2n3wq2zfbttgdpjrj80000gn/T//Rtmpjw3DfT/downloaded_packages


In [15]:
list_of_packages <- c ("synthpop", "insight", "party", "dplyr", "rpart", "rpart.plot", "randomForest", "pROC", "caret", "pracma", "here", "Hmisc", "arf", "randomForest", "caret", "xgboost", "data.table", "tensorflow","keras")

lapply(list_of_packages, FUN= function(X){
  do.call("require", list(X))
})

Lade n"otiges Paket: tensorflow

Warning message:
"Paket 'tensorflow' wurde unter R Version 4.3.2 erstellt"



Attache Paket: 'tensorflow'


Das folgende Objekt ist maskiert 'package:caret':

    train


Lade n"otiges Paket: keras

Warning message:
"Paket 'keras' wurde unter R Version 4.3.2 erstellt"

Attache Paket: 'keras'


Das folgende Objekt ist maskiert 'package:party':

    fit


Das folgende Objekt ist maskiert 'package:modeltools':

    fit


Das folgende Objekt ist maskiert 'package:insight':

    get_weights




[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

[[13]]
[1] TRUE

[[14]]
[1] TRUE

[[15]]
[1] TRUE

[[16]]
[1] TRUE

[[17]]
[1] TRUE

[[18]]
[1] TRUE

[[19]]
[1] TRUE

## Data

In [3]:
load(file = (paste0(here(), "/cpspop.RData")))
adult <- read.csv(file = (paste0(here(),"/adult_preprocessed.csv")))
# delete NAs
adult[adult == "?"] <- NA
adult <- na.omit(adult)

adult$workclass <- as.factor(adult$workclass)
adult$education <- as.factor(adult$education)
adult$marital_status <- as.factor(adult$marital_status)
adult$relationship <- as.factor(adult$relationship)
adult$race <- as.factor(adult$race)
adult$sex <- as.factor(adult$sex)
adult$native_country <- as.factor(adult$native_country)
adult$income <- as.factor(adult$income)

## Trying out the MLP
1. Prepare Data: Split your data into features and target. Normalize the features to ensure better performance of the neural network. <br>
2. Define and Train the MLP Model: Define the architecture of the MLP, compile the model, and train it on your data. <br>
3. Generate Synthetic Data: Sample from the feature space, use the trained model to predict the target variable for these samples, and combine the predictions with the sampled features to create synthetic data.

In [10]:
# convert all variables in df to numeric
convert_to_numeric <- function(df) {
  # Convert factors to numeric
  df[] <- lapply(df, function(col) {
    if (is.factor(col)) {
      return(as.numeric(col))
    } else {
      return(col)
    }
})
  return(df)
}

In [26]:
# Identify features and target
data <- adult

data <- convert_to_numeric(data)
target_col <- 'income'
features <- setdiff(names(data), target_col)

# Split the data into features (X) and target (y)
X <- data[, features]
y <- data[, target_col]

X_means <- colMeans(X)
X_sds <- apply(X, 2, sd)

# Normalize features
X_normalized <- scale(X, center = X_means, scale = X_sds)

# Store the mean and standard deviation of the target
y_mean <- mean(y)
y_sd <- sd(y)

# Normalize target
y_normalized <- scale(y, center = y_mean, scale = y_sd)

In [23]:
# Define the MLP model
model <- keras_model_sequential() %>%
  layer_dense(units = 64, activation = 'relu', input_shape = ncol(X)) %>%
  layer_dense(units = 32, activation = 'relu') %>%
  layer_dense(units = 1)  # Single output for regression

# Compile the model
model %>% compile(
  loss = 'mean_squared_error',
  optimizer = optimizer_adam(),
  metrics = c('mean_absolute_error')
)

# Train the model
history <- model %>% fit(
  as.matrix(X), y,
  epochs = 100,
  batch_size = 32,
  validation_split = 0.2
)

In [27]:
generate_synthetic_data <- function(model, X, X_means, X_sds, y_mean, y_sd, n_samples) {
  synthetic_data <- list()
  for (i in 1:n_samples) {
    # Sample a random row from the normalized feature matrix
    random_sample <- X[sample(1:nrow(X), 1), , drop = FALSE]
    
    # Predict the normalized target variable for the sampled row
    prediction <- model %>% predict(as.matrix(random_sample))
    
    # Denormalize the predicted target
    prediction_denormalized <- prediction * y_sd + y_mean
    
    # Combine the random sample with the denormalized predicted target
    synthetic_sample <- as.data.frame(random_sample)
    synthetic_sample$income <- prediction_denormalized
    
    # Denormalize the features
    denormalized_sample <- synthetic_sample
    for (col in names(synthetic_sample)[-ncol(synthetic_sample)]) {
      denormalized_sample[[col]] <- synthetic_sample[[col]] * X_sds[col] + X_means[col]
    }
    
    # Append to the list
    synthetic_data[[i]] <- denormalized_sample
  }
  
  # Combine all synthetic samples into a data frame
  synthetic_df <- do.call(rbind, synthetic_data)
  return(synthetic_df)
}


In [28]:
# Generate synthetic data
synthetic_data <- generate_synthetic_data(model, X_normalized, X_means, X_sds, y_mean, y_sd, 1000)  # Generate 1000 synthetic samples

print(head(synthetic_data))


      age workclass fnlwgt education marital_status occupation relationship
8343   44         3 193494        16              3         13            1
3701   39         3 328466         2              3          7            1
17407  21         3 186314        12              3          0            1
23435  35         5 114366        10              3          9            1
11688  41         3 277192         5              3          4            6
29080  31         3  73585         9              3          9            1
      race sex capital_gain capital_loss hours_per_week native_country   income
8343     5   2            0            0             72             39 1.137240
3701     5   2         2407            0             70             26 1.173232
17407    5   2            0            0             40             39 1.257761
23435    5   2            0            0             40             39 1.196040
11688    5   1            0            0             40             